In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
import random
from itertools import chain

import create_test_data
import create_gene
import calculate_fitness
import select_mating_pool
import crossover

from tqdm import tqdm

rng = default_rng()

In [5]:
# create test subjects and test populations

n_people = 150 # number of people

n_genes = 2000 # number of genes (population)

n_iterations = 200

companies = {'Zeix': 20, 'SCS': 20, 'Kyburz': 20, 'Siemens': 20, 'Varian': 20, 'MAN':20, 'USZ': 20, 
             'Acht Grad Ost': 20, 'Weidmann': 20, 'True Wealth': 20, 'PartnerRe': 20, 'Sensirion': 20, 'Hocoma': 20}
uni1 = {'Architektur': 30, 'Biologie (ETH)': 60, 'Raum (ETH)': 16, 'Umwelt (ETH)': 16, 
        'Drohnen (UZH)': 40, 'Biodiversität (UZH)': 24, 'Physik (UZH)': 20}
uni2 = {'Chemie (ETH)': 25, 'Material (ETH)': 20, 'Medizin (ETH)': 50, 'Pharma (ETH)': 24, 'Physik (ETH)': 20,
        'Chemie (UZH)': 30, 'Informatik (UZH)': 20}
uni3 = {'CERN (UZH)': 30, 'Hyperwürfel (UZH)': 20, 'Krebsforschung (UZH)': 20, 
        'Informatik (ETH)': 16, 'Elektrotechnik (ETH)': 30, 'Maschinen (ETH)': 40, 'Mathematik (ETH)': 30}
workshops = {'Oberflächen und Farben': 20, 'Fourier': 20, 'Datenexploration': 20, 'Astrophysik': 20,
             'Molekulare Schalter': 12, 'Arduino': 12, 'Erneuerbar': 20, 'SYPT': 20, 'Big Data': 16, 
             'DNA': 20, 'Astronomie': 20, 'Recycling': 16, 'Molekulargenetik': 14, 'Energiespeicherung': 20}

blocks = {'companies': companies, 'uni1': uni1, 'uni2': uni2, 'uni3': uni3, 'workshops': workshops}
block_names = [block for block in blocks.keys()]

multipliers = [50, 2, 1]
counter = [0, 0, 0]

choices_df = create_test_data.get_choices(blocks, n_people)

print('Generating initial population:')
genes = {}
for i in tqdm(range(n_genes)):
    id = f'GID{i:05}'
    gene_df = create_gene.get_gene(blocks, n_people)

    fitness = calculate_fitness.calc_fitness(choices_df, gene_df, block_names, multipliers)

    genes[id] = [gene_df, fitness]


frac_elite = 0.1
frac_lucky = 0.005

print('Optimising population:')
for i in tqdm(range(n_iterations)):
    pool = select_mating_pool.get_mating_pool(genes, frac_elite, frac_lucky)

    offsprings = crossover.get_offspring(pool, choices_df, blocks, n_genes, multipliers)

    genes = offsprings


# print(np.mean([genes[g][1] for g in genes]))

best = genes[select_mating_pool.get_best(genes)][0]

first = 0
second = 0
third = 0

for block in blocks:
        
    first = (choices_df[f'{block}_first_choice'] == best[f'{block}']).sum()

    for c, g, s in zip(choices_df[f'{block}_second_choices'], best[f'{block}'], choices_df['total_score']):
        if g in c:
            second += 1

    for c, g, s in zip(choices_df[f'{block}_third_choices'], best[f'{block}'], choices_df['total_score']):
        if g in c:
            third += 1

    print(block)
    print(f'{first = }, {second = }, {third = }')

Generating initial population:


100%|██████████| 2000/2000 [00:05<00:00, 379.16it/s]


Optimising population:


100%|██████████| 200/200 [20:02<00:00,  6.01s/it]

companies
first = 3, second = 83, third = 45
uni1
first = 17, second = 156, third = 73
uni2
first = 7, second = 227, third = 121
uni3
first = 7, second = 308, third = 152
workshops
first = 150, second = 308, third = 152


In [2]:
companies = {'Zeix': 20, 'SCS': 20, 'Kyburz': 20, 'Siemens': 20, 'Varian': 20, 'MAN':20, 'USZ': 20, 
             'Acht Grad Ost': 20, 'Weidmann': 20, 'True Wealth': 20, 'PartnerRe': 20, 'Sensirion': 20, 'Hocoma': 20}
uni1 = {'Architektur': 30, 'Biologie (ETH)': 60, 'Raum (ETH)': 16, 'Umwelt (ETH)': 16, 
        'Drohnen (UZH)': 40, 'Biodiversität (UZH)': 24, 'Physik (UZH)': 20}
uni2 = {'Chemie (ETH)': 25, 'Material (ETH)': 20, 'Medizin (ETH)': 50, 'Pharma (ETH)': 24, 'Physik (ETH)': 20,
        'Chemie (UZH)': 30, 'Informatik (UZH)': 20}
uni3 = {'CERN (UZH)': 30, 'Hyperwürfel (UZH)': 20, 'Krebsforschung (UZH)': 20, 
        'Informatik (ETH)': 16, 'Elektrotechnik (ETH)': 30, 'Maschinen (ETH)': 40, 'Mathematik (ETH)': 30}
workshops = {'Oberflächen und Farben': 20, 'Fourier': 20, 'Datenexploration': 20, 'Astrophysik': 20,
             'Molekulare Schalter': 12, 'Arduino': 12, 'Erneuerbar': 20, 'SYPT': 20, 'Big Data': 16, 
             'DNA': 20, 'Astronomie': 20, 'Recycling': 16, 'Molekulargenetik': 14, 'Energiespeicherung': 20}

blocks = {'companies': companies, 'uni1': uni1, 'uni2': uni2, 'uni3': uni3, 'workshops': workshops}
block_names = [block for block in blocks.keys()]

In [5]:
n_people = 10
multipliers = [50, 5, 1]
choices_df = create_test_data.get_choices(blocks, n_people)
gene_df = create_gene.get_gene(blocks, n_people)
fitness = calculate_fitness.calc_fitness(choices_df, gene_df, block_names, multipliers)

In [6]:
gene_df

,companies,uni1,uni2,uni3,workshops
ID000,SCS,Biologie (ETH),Material (ETH),Maschinen (ETH),SYPT
ID001,MAN,Umwelt (ETH),Medizin (ETH),Elektrotechnik (ETH),Erneuerbar
ID002,Varian,Architektur,Medizin (ETH),CERN (UZH),Fourier
ID003,Weidmann,Biologie (ETH),Chemie (UZH),CERN (UZH),Recycling
ID004,USZ,Physik (UZH),Material (ETH),Maschinen (ETH),Datenexploration
ID005,Zeix,Physik (UZH),Medizin (ETH),Informatik (ETH),DNA
ID006,MAN,Architektur,Pharma (ETH),CERN (UZH),Big Data
ID007,Varian,Biologie (ETH),Pharma (ETH),Maschinen (ETH),Energiespeicherung
ID008,Weidmann,Biologie (ETH),Informatik (UZH),Maschinen (ETH),Arduino
ID009,Kyburz,Biologie (ETH),Informatik (UZH),CERN (UZH),Astronomie


In [7]:
fitness

8140